In [1]:
import os
import pickle

import numpy as np
import pandas as pd
import torch

import matplotlib.pyplot as plt

from torch.utils.data import DataLoader, random_split
from torch.nn.functional import one_hot

from data_loaders.pulja_data_loader_02 import PuljaDataLoader
from models._20220729_00 import UserModel
from models.utils_02 import collate_fn

if torch.cuda.is_available():
    from torch.cuda import FloatTensor, LongTensor
    torch.set_default_tensor_type(torch.cuda.FloatTensor)
else:
    from torch import FloatTensor, LongTensor

In [2]:
ckpt_path = "ckpts"
if not os.path.isdir(ckpt_path):
    os.mkdir(ckpt_path)

batch_size = 256
num_epochs = 100
train_ratio = 0.9

seq_len = 1000

dim_v = 20

dataset = PuljaDataLoader(seq_len)

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

model = UserModel(dataset.num_c1, dataset.num_c2, dataset.num_c3, dataset.num_d, dim_v).to(device)

train_size = int(len(dataset) * train_ratio)
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(
    dataset, [train_size, test_size]
)

if os.path.exists(os.path.join(dataset.dataset_dir, "indices.pkl")):
    with open(os.path.join(dataset.dataset_dir, "indices.pkl"), "rb") as f:
        indices = pickle.load(f)
    train_dataset.indices = indices[0]
    test_dataset.indices = indices[1]
else:
    with open(os.path.join(dataset.dataset_dir, "indices.pkl"), "wb") as f:
        pickle.dump((train_dataset.indices, test_dataset.indices), f)


In [3]:
model.load_state_dict(
    torch.load(
        os.path.join(
            os.path.join(
                ckpt_path, "_20220729_00"
            ),
            "model_fin.ckpt"
        ),
        map_location=device
    )
)
model.eval()

UserModel(
  (D): Embedding(7, 1)
  (R): Embedding(2, 20)
  (gru): GRU(40, 20, batch_first=True)
  (linear_1): Sequential(
    (0): Linear(in_features=20, out_features=20, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=20, out_features=1, bias=True)
    (4): Dropout(p=0.5, inplace=False)
  )
  (linear_2): Sequential(
    (0): Linear(in_features=60, out_features=20, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=20, out_features=1, bias=True)
    (4): Dropout(p=0.5, inplace=False)
  )
)

In [4]:
c3 = 0

d = np.random.choice(5)

h = torch.zeros([1, model.dim_v])
C3 = torch.zeros([1, model.num_c3])

alpha = model.linear_1(h).squeeze()

beta3 = torch.gather(
    C3, dim=-1, index=LongTensor([[c3]])
).squeeze()

gamma = model.D(LongTensor([[d]])).squeeze()

logit = alpha + beta3 - gamma

p = torch.sigmoid(logit).detach().cpu().numpy()

r = np.random.binomial(1, p)

print("---------------------------")
print(alpha, alpha - gamma, d, r)
print(C3[0, 0])

for _ in range(30):
    alpha, h, C3 = model(
        LongTensor([[c3]]),
        LongTensor([[d]]),
        LongTensor([[r]]),
        alpha,
        h,
        C3,
    )
    
    alpha = alpha.squeeze()
    h = h[0]
    C3 = C3[0]
    
    d = np.where((alpha.detach().cpu().numpy() - model.D.weight.squeeze().detach().cpu().numpy() < 0)[:-1])[0][0]
    
    beta3 = torch.gather(
        C3, dim=-1, index=LongTensor([[c3]])
    ).squeeze()

    gamma = model.D(LongTensor([[d]])).squeeze()

    logit = alpha + beta3 - gamma

    p = torch.sigmoid(logit).detach().cpu().numpy()

    r = np.random.binomial(1, p)
    
    print("---------------------------")
    print(alpha, alpha - gamma, d, r)
    print(C3[0, 0])

---------------------------
tensor(0.2150, grad_fn=<SqueezeBackward0>) tensor(1.4039, grad_fn=<SubBackward0>) 0 1
tensor(0.)
sssssssssss
tensor([0.3612], grad_fn=<ViewBackward>)
---------------------------
tensor(0.2150, grad_fn=<SqueezeBackward0>) tensor(-0.0284, grad_fn=<SubBackward0>) 3 1
tensor(0.1358, grad_fn=<SelectBackward>)
sssssssssss
tensor([0.2709], grad_fn=<ViewBackward>)
---------------------------
tensor(0.2709, grad_fn=<SqueezeBackward0>) tensor(-0.4124, grad_fn=<SubBackward0>) 4 1
tensor(0.1097, grad_fn=<SelectBackward>)
sssssssssss
tensor([0.3662], grad_fn=<ViewBackward>)
---------------------------
tensor(0.3662, grad_fn=<SqueezeBackward0>) tensor(-0.3171, grad_fn=<SubBackward0>) 4 0
tensor(0.0843, grad_fn=<SelectBackward>)
sssssssssss
tensor([0.4481], grad_fn=<ViewBackward>)
---------------------------
tensor(0.3662, grad_fn=<SqueezeBackward0>) tensor(-0.3171, grad_fn=<SubBackward0>) 4 1
tensor(-0.0370, grad_fn=<SelectBackward>)
sssssssssss
tensor([0.3306], grad_fn=<

In [5]:
gamma

tensor(0.6833, grad_fn=<SqueezeBackward0>)

In [6]:
beta3

tensor(0.0449, grad_fn=<SqueezeBackward0>)

In [7]:
alpha

tensor(0.3662, grad_fn=<SqueezeBackward0>)

In [8]:
logit

tensor(-0.2722, grad_fn=<SubBackward0>)

In [9]:
p

array(0.43236467, dtype=float32)

In [10]:
r

0

In [11]:
end

NameError: name 'end' is not defined

In [ ]:
alpha_seq, h_seq, C3_seq = model(
    LongTensor([[0]]),
    LongTensor([[d]]),
    LongTensor(r_seq),
)

In [ ]:
test_dataset[47]

In [ ]:
# 3, 5, 6, 7, 11, 12, 15, 20
c3_seq, d_seq, r_seq = test_dataset[12]

c3_seq = [c3_seq[:seq_len]]
d_seq = [d_seq[:seq_len]]
r_seq = [r_seq[:seq_len]]

# r_seq = np.zeros_like(r_seq)
# r_seq = np.ones_like(r_seq)

# d_seq = np.ones_like(d_seq) * 6

# c_seq = np.array([[0] * 100])
# d_seq = np.array([[6] * 30 + [3] * 30 + [0] * 40])
# r_seq = np.ones_like(r_seq)

# c1_seq, c2_seq, c4_seq, d_seq, r_seq: [batch_size, seq_len]

alpha_seq, h_seq, C3_seq = model(
    LongTensor(c3_seq),
    LongTensor(d_seq),
    LongTensor(r_seq),
)

# C3_seq: [batch_size, seq_len, num_c4]

# beta3_seq: [batch_size, seq_len]
beta3_seq = torch.gather(C3_seq, dim=-1, index=LongTensor(c3_seq).unsqueeze(-1)).reshape([1, seq_len])

# gamma_seq: [seq_len]
gamma_seq = model.D(LongTensor(d_seq)).squeeze()

###########################################

sigmoid_alpha_seq = torch.sigmoid(alpha_seq).detach().cpu().numpy()[0]

sigmoid_alpha_beta_seq = torch.sigmoid(alpha_seq + beta3_seq).detach().cpu().numpy()[0]

sigmoid_alpha_beta_gamma_seq = torch.sigmoid(alpha_seq + beta3_seq - gamma_seq).detach().cpu().numpy()[0]

r_seq_ma = np.cumsum(r_seq[0]) / np.arange(1, r_seq[0].shape[0] + 1)

CQ3_seq = alpha_seq.unsqueeze(-1) + C3_seq.squeeze()
CQ3_seq = torch.sigmoid(CQ3_seq).detach().cpu().numpy()[0]

# C3_seq = C3_seq.squeeze().detach().cpu().numpy()

alpha_seq = alpha_seq.detach().cpu().numpy()
gamma_seq = gamma_seq.detach().cpu().numpy()

prev_alpha = alpha_seq[0][:-1]
prev_alpha = np.concatenate([model.init_alpha(1).detach().cpu().numpy(), prev_alpha])

In [ ]:
view_from = 0
view_range = 200

for view_from in range(0, seq_len, view_range):
    view_to = view_from + view_range

    plt.figure(figsize=[15, 5], facecolor="white")
    # plt.figure(figsize=[100, 10], facecolor="white")
    plt.plot(r_seq[0][view_from:view_to], label="r_seq")
    plt.plot(((d_seq[0] + 1) / 10)[view_from:view_to], label="d_seq")
    plt.plot(sigmoid_alpha_seq[view_from:view_to], label="sigmoid_alpha_seq")
    plt.plot(sigmoid_alpha_beta_seq[view_from:view_to], label="sigmoid_alpha_beta_seq")
    # plt.plot(sigmoid_alpha_beta4_seq, label="sigmoid_alpha_beta4_seq")
    # plt.plot(sigmoid_beta4_seq, label="sigmoid_beta4_seq")
    plt.plot(((prev_alpha - gamma_seq) / 10)[view_from:view_to], label="alpha - gamma")
    plt.grid()
    plt.ylim(-0.1, 1.1)
    plt.legend()
    plt.show()

plt.figure(figsize=[20, len(np.unique(c3_seq)) // 3 + 1], facecolor="white")
temp = np.concatenate([CQ3_seq[:, np.unique(c3_seq)].T, r_seq], axis=0)
plt.pcolormesh(temp)
plt.colorbar()
plt.show()

In [ ]:
CQ3_seq[0][np.unique(c3_seq)]

In [ ]:
CQ3_seq[-1][np.unique(c3_seq)]

In [ ]:
np.unique(c3_seq), c3_seq

In [ ]:
dataset.c3_list[np.unique(c3_seq)]

In [ ]:
dataset.c3_list[np.unique(c3_seq)[np.argmin(CQ3_seq[-1][np.unique(c3_seq)])]]

In [ ]:
dataset.c32c4_list[dataset.c3_list[np.unique(c3_seq)[np.argmin(CQ3_seq[-1][np.unique(c3_seq)])]]]

In [ ]:
d_seq

In [ ]:
model.D.weight

In [ ]:
model.init_alpha